# Front End

# Back End

In [84]:
from PIL import Image
from inference_sdk import InferenceHTTPClient

In [85]:
image_to_test = "base_station.jpg"

In [86]:
image = Image.open(image_to_test)

In [87]:
# initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="<api_key>"
)

# infer on a local image
result = CLIENT.infer(image_to_test, model_id="<model_id>")

In [88]:
result

{'inference_id': '30105660-9aec-473b-9669-8b327036eb67',
 'time': 0.03940387000056944,
 'image': {'width': 592, 'height': 481},
 'predictions': [{'x': 458.5,
   'y': 114.5,
   'width': 163.0,
   'height': 35.0,
   'confidence': 0.8475237488746643,
   'class': 'car-plates',
   'class_id': 1,
   'detection_id': '9e114176-f329-47b9-bad3-0657cef7ae93'},
  {'x': 217.5,
   'y': 187.0,
   'width': 199.0,
   'height': 40.0,
   'confidence': 0.8441112041473389,
   'class': 'car-plates',
   'class_id': 1,
   'detection_id': '8b9acf30-ad00-4b32-85da-c59c5bae9fbd'},
  {'x': 193.5,
   'y': 365.5,
   'width': 45.0,
   'height': 169.0,
   'confidence': 0.8401526212692261,
   'class': 'car-plates',
   'class_id': 1,
   'detection_id': 'daca8deb-486d-4231-a046-af16c745f212'},
  {'x': 195.0,
   'y': 435.0,
   'width': 28.0,
   'height': 18.0,
   'confidence': 0.8263109922409058,
   'class': 'N',
   'class_id': 0,
   'detection_id': '657d215f-aaf9-4749-b39f-9dd55edc7518'},
  {'x': 465.0,
   'y': 388.5,
 

In [89]:
number_of_plates = 0

for item in result['predictions']:

    if item['class_id'] == 1:
# Count the total number of plates in the image
        number_of_plates = number_of_plates + 1
# Setting the points for cropped image
        left = item['x'] - item['width']/2 
        top = item['y'] - item['height']/2 
        right = item['x'] + item['width']/2 
        bottom = item['y'] + item['height']/2
 
# Cropped image of above dimension
# (It will not change original image)
        im = image.crop((left, top, right, bottom))
# If the plate image is too small resize it
        if min(im.size) < 60:
                if im.size[0] == min(im.size):
                        new_im = im.resize((50, int(im.size[1]*60/im.size[0] + 1)))
                else:
                        new_im = im.resize((int(im.size[0]*60/im.size[1] + 1), 60))
                new_im.save('to_ocr_' + str(number_of_plates) + '.jpg')
        else:
                im.save('to_ocr_' + str(number_of_plates) + '.jpg')
       
 
# Shows the image in image viewer
        im.show()

In [ ]:
# Try pytesseract - not stable
#import pytesseract
#text = []

#while number_of_plates > 0:
#    text.append(pytesseract.image_to_string('to_ocr_' + str(number_of_plates) +'.jpg'))
#    number_of_plates = number_of_plates - 1


In [ ]:
# text

In [90]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
import time

In [91]:
region = "<region>"
key = "<access key>"
numberOfCharsInOperationId = 36


credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://<myendpoint>.cognitiveservices.azure.com/",
    credentials=credentials
)

In [92]:
license_plates = []
while number_of_plates > 0:
    file_name = 'to_ocr_' + str(number_of_plates) + '.jpg'
    read_image = open(file_name, "rb")
    rawHttpResponse = client.read_in_stream(read_image, language="en", raw=True)
    operationLocation = rawHttpResponse.headers["Operation-Location"]
    idLocation = len(operationLocation) - numberOfCharsInOperationId
    operationId = operationLocation[idLocation:]
    while True:
    # get_read_result() is asynchronous, need to check when it finishes
        result = client.get_read_result(operationId)
        if result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)
    if result.status == OperationStatusCodes.succeeded:
        for line in result.analyze_result.read_results[0].lines:
            if(len(line.text) > 5):
                print(line.text)
                license_plates.append(line.text)
    number_of_plates =  number_of_plates - 1

ZM 78528
LY 66972
BP147398
PP 90693
EB 74699


In [93]:
license_plates

['ZM 78528', 'LY 66972', 'BP147398', 'PP 90693', 'EB 74699']